In [ ]:
## This file implements neural networks before and after lasso selection for p0017presabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017presabsSTCC_qual.csv')
df.shape

(253, 7159)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    0
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_4749,group_6727,group_8892,group_9205,group_9474,group_9475,group_9858,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
3,120335,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,5,5,2
4,120337,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,5,5,2


In [6]:
df['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 7158)

In [9]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,...,group_4749,group_6727,group_8892,group_9205,group_9474,group_9475,group_9858,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,5,5,2
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,5,5,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 7158) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
112,NRS027,1
207,NRS246,1
186,NRS218,2
88,CFBRSa70,2
156,NRS177,1
...,...,...
244,SR3585,0
99,GA50819,2
147,NRS161,2
49,CFBREBSa114,1


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 950us/step - loss: 1.3886 - accuracy: 0.4181 - val_loss: 1.6842 - val_accuracy: 0.4211
Epoch 2/100
177/177 [==============================] - 0s 400us/step - loss: 1.6026 - accuracy: 0.4972 - val_loss: 1.5356 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 370us/step - loss: 1.3812 - accuracy: 0.5141 - val_loss: 1.0383 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 379us/step - loss: 1.2332 - accuracy: 0.5763 - val_loss: 1.2765 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 412us/step - loss: 1.1373 - accuracy: 0.6215 - val_loss: 1.0610 - val_accuracy: 0.4211
Epoch 6/100
177/177 [==============================] - 0s 464us/step - loss: 0.8628 - accuracy: 0.6328 - val_loss: 1.2545 - val_accuracy: 0.4737
Epoch 7/100
177/177 [==============================] - 0s 468us/step - loss: 0.8375 -

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.103770). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 959us/step - loss: 0.4124 - accuracy: 0.9096 - val_loss: 1.9522 - val_accuracy: 0.4342
Epoch 47/100
177/177 [==============================] - 0s 373us/step - loss: 0.4175 - accuracy: 0.9266 - val_loss: 2.1623 - val_accuracy: 0.5132
Epoch 48/100
177/177 [==============================] - 0s 904us/step - loss: 0.3777 - accuracy: 0.8531 - val_loss: 1.8624 - val_accuracy: 0.4868
Epoch 49/100
177/177 [==============================] - 0s 695us/step - loss: 0.3646 - accuracy: 0.8983 - val_loss: 1.9232 - val_accuracy: 0.4605
Epoch 50/100
177/177 [==============================] - 0s 370us/step - loss: 0.2664 - accuracy: 0.9492 - val_loss: 1.9088 - val_accuracy: 0.5000
Epoch 51/100
177/177 [==============================] - 0s 372us/step - loss: 0.2606 - accuracy: 0.9492 - val_loss: 1.8684 - val_accuracy: 0.5000
Epoch 52/100
177/177 [==============================] - 0s 357us/step - loss: 0.2305 - accuracy: 0.9548 - val_loss: 1.9589 - val_accuracy

In [43]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 167us/step
test accuracy: 48.68%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 1, 2, 2, 0, 0, 1, 2, 1, 2, 1, 0,
       2, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 1, 2, 2, 0, 2, 0, 0, 1, 0, 2, 0,
       2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1,
       0, 2, 1, 1, 0, 0, 0, 0, 0, 2])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
112,NRS027,1,0
207,NRS246,1,0
186,NRS218,2,1
88,CFBRSa70,2,0
156,NRS177,1,1
...,...,...,...
244,SR3585,0,0
99,GA50819,2,0
147,NRS161,2,0
49,CFBREBSa114,1,0


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,0.748608,0.091089,0.160304
1,0.689107,0.292688,0.018205
2,0.029113,0.924627,0.046260
3,0.933857,0.033455,0.032688
4,0.032013,0.939939,0.028048
...,...,...,...
71,0.926324,0.052714,0.020962
72,0.949648,0.014513,0.035839
73,0.755204,0.244795,0.000001
74,0.989175,0.001360,0.009465


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17pST.csv", index = False,
         header=None)

In [47]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 445us/step - loss: 0.1171 - accuracy: 0.9548 - val_loss: 2.2946 - val_accuracy: 0.4737
Epoch 2/100
177/177 [==============================] - 0s 354us/step - loss: 0.1351 - accuracy: 0.9605 - val_loss: 2.2022 - val_accuracy: 0.5000
Epoch 3/100
177/177 [==============================] - 0s 385us/step - loss: 0.1407 - accuracy: 0.9661 - val_loss: 2.3433 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 344us/step - loss: 0.1321 - accuracy: 0.9718 - val_loss: 2.4889 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 399us/step - loss: 0.1166 - accuracy: 0.9944 - val_loss: 2.2457 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 375us/step - loss: 0.1167 - accuracy: 0.9774 - val_loss: 2.2751 - val_accuracy: 0.5132
Epoch 7/100
177/177 [==============================] - 0s 376us/step - loss: 0.1096 -

Epoch 57/100
177/177 [==============================] - 0s 607us/step - loss: 0.0742 - accuracy: 0.9831 - val_loss: 2.8205 - val_accuracy: 0.4079
Epoch 58/100
177/177 [==============================] - 0s 1ms/step - loss: 0.0764 - accuracy: 0.9831 - val_loss: 2.7541 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 850us/step - loss: 0.0764 - accuracy: 0.9774 - val_loss: 2.7426 - val_accuracy: 0.4868
Epoch 60/100
177/177 [==============================] - 0s 611us/step - loss: 0.1015 - accuracy: 0.9887 - val_loss: 2.6501 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 582us/step - loss: 0.1170 - accuracy: 0.9831 - val_loss: 2.6542 - val_accuracy: 0.4737
Epoch 62/100
177/177 [==============================] - 0s 371us/step - loss: 0.1002 - accuracy: 0.9887 - val_loss: 2.8594 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 381us/step - loss: 0.0949 - accuracy: 0.9718 - val_loss: 2.7059 - v

In [48]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 97.94%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.48607600e-01, 9.10885860e-02, 1.60303830e-01],
       [6.89107400e-01, 2.92687540e-01, 1.82050450e-02],
       [2.91128890e-02, 9.24627300e-01, 4.62598430e-02],
       [9.33857260e-01, 3.34546680e-02, 3.26880330e-02],
       [3.20128540e-02, 9.39938800e-01, 2.80482900e-02],
       [9.89545300e-01, 5.98503600e-03, 4.46960140e-03],
       [9.70200400e-01, 1.17197650e-02, 1.80798840e-02],
       [9.94562570e-01, 1.24398170e-05, 5.42505700e-03],
       [8.66072000e-01, 1.16183780e-01, 1.77441820e-02],
       [3.48143570e-04, 8.43306200e-02, 9.15321230e-01],
       [5.55978700e-02, 9.33403800e-01, 1.09983780e-02],
       [7.74316200e-03, 9.84019640e-01, 8.23720100e-03],
       [2.75979980e-03, 6.48523100e-02, 9.32387900e-01],
       [2.96608490e-02, 9.57406600e-02, 8.74598440e-01],
       [5.28993070e-01, 2.02217500e-01, 2.68789470e-01],
       [9.57669260e-01, 4.22977170e-02, 3.30163400e-05],
       [1.23803560e-02, 9.67566500e-01, 2.00532040e-02],
       [1.28446060e-07, 5.16763

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [21]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.6532530520625758

In [22]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.6532530520625758

In [24]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [25]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [26]:
dat2

,id,test
178,NRS210,2
104,Grady1,0
79,CFBRSa29,2
66,CFBRSa03,0
9,217,1
...,...,...
25,BCH-SA-10,1
94,GA15,2
244,SR3585,0
227,NRS387,2


In [27]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [28]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.9756 - accuracy: 0.3672 - val_loss: 1.1070 - val_accuracy: 0.4737
Epoch 2/100
177/177 [==============================] - 0s 803us/step - loss: 1.1481 - accuracy: 0.4237 - val_loss: 0.9961 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 720us/step - loss: 1.0713 - accuracy: 0.5537 - val_loss: 0.9700 - val_accuracy: 0.5395
Epoch 4/100
177/177 [==============================] - 0s 762us/step - loss: 1.0669 - accuracy: 0.5537 - val_loss: 1.4252 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 812us/step - loss: 0.9918 - accuracy: 0.5932 - val_loss: 0.9314 - val_accuracy: 0.5263
Epoch 6/100
177/177 [==============================] - 0s 629us/step - loss: 0.9061 - accuracy: 0.6328 - val_loss: 1.1522 - val_accuracy: 0.4474
Epoch 7/100
177/177 [==============================] - 0s 614us/step - loss: 0.8754 - a

Epoch 57/100
177/177 [==============================] - 0s 466us/step - loss: 0.3237 - accuracy: 0.8814 - val_loss: 1.8147 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 452us/step - loss: 0.2528 - accuracy: 0.9153 - val_loss: 1.7715 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 494us/step - loss: 0.2654 - accuracy: 0.8870 - val_loss: 1.7424 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 469us/step - loss: 0.2468 - accuracy: 0.8983 - val_loss: 1.7738 - val_accuracy: 0.4737
Epoch 61/100
177/177 [==============================] - 0s 600us/step - loss: 0.2319 - accuracy: 0.9209 - val_loss: 1.8342 - val_accuracy: 0.4474
Epoch 62/100
177/177 [==============================] - 0s 597us/step - loss: 0.2296 - accuracy: 0.9322 - val_loss: 1.8496 - val_accuracy: 0.4474
Epoch 63/100
177/177 [==============================] - 0s 600us/step - loss: 0.2362 - accuracy: 0.9266 - val_loss: 1.7988 -

In [76]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 212us/step
test accuracy: 50.00%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([1, 2, 0, 1, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 0, 2, 0, 1, 0, 1, 0, 2, 1, 2, 0,
       0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 0,
       2, 1, 2, 2, 0, 1, 2, 0, 2, 1])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
178,NRS210,2,1
104,Grady1,0,2
79,CFBRSa29,2,0
66,CFBRSa03,0,1
9,217,1,1
...,...,...,...
25,BCH-SA-10,1,1
94,GA15,2,2
244,SR3585,0,0
227,NRS387,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,0.081527,0.918088,0.000385
1,0.329519,0.304125,0.366356
2,0.632590,0.260623,0.106787
3,0.011812,0.975836,0.012352
4,0.363130,0.447296,0.189574
...,...,...,...
71,0.018751,0.640052,0.341197
72,0.355005,0.229513,0.415482
73,0.748998,0.142183,0.108819
74,0.058424,0.101928,0.839647


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17pST.csv", index = False,
         header=None)

In [80]:
hist2 = model2.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 670us/step - loss: 0.1932 - accuracy: 0.9266 - val_loss: 2.2315 - val_accuracy: 0.4605
Epoch 2/100
177/177 [==============================] - 0s 549us/step - loss: 0.2160 - accuracy: 0.9266 - val_loss: 2.2207 - val_accuracy: 0.5526
Epoch 3/100
177/177 [==============================] - 0s 523us/step - loss: 0.1843 - accuracy: 0.9266 - val_loss: 2.2431 - val_accuracy: 0.5132
Epoch 4/100
177/177 [==============================] - 0s 498us/step - loss: 0.2308 - accuracy: 0.9322 - val_loss: 2.2242 - val_accuracy: 0.5526
Epoch 5/100
177/177 [==============================] - 0s 531us/step - loss: 0.2139 - accuracy: 0.9322 - val_loss: 2.2155 - val_accuracy: 0.5263
Epoch 6/100
177/177 [==============================] - 0s 525us/step - loss: 0.1950 - accuracy: 0.9209 - val_loss: 2.2539 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 593us/step - loss: 0.1861 -

Epoch 57/100
177/177 [==============================] - 0s 521us/step - loss: 0.1497 - accuracy: 0.9548 - val_loss: 2.7127 - val_accuracy: 0.5000
Epoch 58/100
177/177 [==============================] - 0s 465us/step - loss: 0.1414 - accuracy: 0.9548 - val_loss: 2.7385 - val_accuracy: 0.4737
Epoch 59/100
177/177 [==============================] - 0s 465us/step - loss: 0.1594 - accuracy: 0.9435 - val_loss: 2.7252 - val_accuracy: 0.4868
Epoch 60/100
177/177 [==============================] - 0s 472us/step - loss: 0.2485 - accuracy: 0.9379 - val_loss: 2.7385 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 479us/step - loss: 0.1486 - accuracy: 0.9379 - val_loss: 2.7181 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 489us/step - loss: 0.3784 - accuracy: 0.9040 - val_loss: 2.7622 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 487us/step - loss: 0.5062 - accuracy: 0.8192 - val_loss: 2.7539 -

In [81]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 93.55%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[8.15265400e-02, 9.18088140e-01, 3.85301160e-04],
       [3.29518850e-01, 3.04124650e-01, 3.66356500e-01],
       [6.32589700e-01, 2.60622950e-01, 1.06787370e-01],
       [1.18123260e-02, 9.75835740e-01, 1.23519290e-02],
       [3.63130150e-01, 4.47296000e-01, 1.89573880e-01],
       [5.49466400e-03, 3.60497200e-01, 6.34008100e-01],
       [9.59864900e-01, 1.86675860e-02, 2.14674580e-02],
       [1.40141710e-01, 6.41925700e-01, 2.17932600e-01],
       [3.00830860e-08, 2.22404250e-04, 9.99777600e-01],
       [9.02642500e-01, 5.99486500e-02, 3.74088180e-02],
       [3.25059700e-01, 5.53053440e-01, 1.21886870e-01],
       [2.72419040e-05, 4.67871000e-02, 9.53185700e-01],
       [7.96510200e-01, 2.01096580e-01, 2.39321820e-03],
       [7.56294800e-02, 9.19433100e-01, 4.93738900e-03],
       [7.91458900e-02, 8.85281700e-01, 3.55723460e-02],
       [3.38485420e-01, 6.34423730e-01, 2.70908180e-02],
       [1.09793840e-04, 9.99885560e-01, 4.66086300e-06],
       [2.17132370e-01, 7.80801

In [32]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.5909205433014956

In [33]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.5909205433014956

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [35]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [36]:
dat3

,id,test
231,NY360,2
92,EUH25,2
91,EUH15,2
203,NRS241,0
242,SR2852,2
...,...,...
26,BCH-SA-11,0
111,NRS022,1
96,GA27,2
129,NRS102,1


In [37]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [43]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 2ms/step - loss: 1.7500 - accuracy: 0.3503 - val_loss: 1.3662 - val_accuracy: 0.3684
Epoch 2/100
177/177 [==============================] - 0s 683us/step - loss: 1.1352 - accuracy: 0.5537 - val_loss: 1.6999 - val_accuracy: 0.3026
Epoch 3/100
177/177 [==============================] - 0s 713us/step - loss: 1.0572 - accuracy: 0.5593 - val_loss: 1.4395 - val_accuracy: 0.3684
Epoch 4/100
177/177 [==============================] - 0s 883us/step - loss: 0.8711 - accuracy: 0.6102 - val_loss: 1.3950 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 642us/step - loss: 0.7997 - accuracy: 0.6497 - val_loss: 1.4311 - val_accuracy: 0.4342
Epoch 6/100
177/177 [==============================] - 0s 600us/step - loss: 0.8549 - accuracy: 0.6384 - val_loss: 1.4501 - val_accuracy: 0.4211
Epoch 7/100
177/177 [==============================] - 0s 586us/step - loss: 0.9850 - a

Epoch 57/100
177/177 [==============================] - 0s 589us/step - loss: 0.2208 - accuracy: 0.9379 - val_loss: 2.2204 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 697us/step - loss: 0.2013 - accuracy: 0.9322 - val_loss: 2.2654 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 738us/step - loss: 0.2190 - accuracy: 0.9322 - val_loss: 2.2747 - val_accuracy: 0.4342
Epoch 60/100
177/177 [==============================] - 0s 508us/step - loss: 0.2126 - accuracy: 0.9322 - val_loss: 2.2106 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 488us/step - loss: 0.2412 - accuracy: 0.9209 - val_loss: 2.4299 - val_accuracy: 0.4605
Epoch 62/100
177/177 [==============================] - 0s 631us/step - loss: 0.2358 - accuracy: 0.9096 - val_loss: 2.3668 - val_accuracy: 0.4474
Epoch 63/100
177/177 [==============================] - 0s 471us/step - loss: 0.1971 - accuracy: 0.9266 - val_loss: 2.3116 -

In [109]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 226us/step
test accuracy: 44.74%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([1, 2, 2, 1, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 2,
       0, 2, 1, 2, 1, 1, 0, 1, 2, 0, 1, 2, 2, 1, 2, 2, 0, 1, 1, 1, 0, 2,
       0, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 2,
       2, 0, 2, 2, 1, 0, 2, 2, 0, 0])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
231,NY360,2,1
92,EUH25,2,2
91,EUH15,2,2
203,NRS241,0,1
242,SR2852,2,2
...,...,...,...
26,BCH-SA-11,0,0
111,NRS022,1,2
96,GA27,2,2
129,NRS102,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.126397,0.865422,0.008181
1,0.410383,0.000640,0.588977
2,0.029125,0.018543,0.952332
3,0.202512,0.744713,0.052775
4,0.000277,0.052598,0.947125
...,...,...,...
71,0.909264,0.084855,0.005880
72,0.005166,0.001040,0.993794
73,0.000167,0.043086,0.956747
74,0.903737,0.092407,0.003856


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17pST.csv", index = False,
         header=None)

In [113]:
hist3 = model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 706us/step - loss: 0.1394 - accuracy: 0.9718 - val_loss: 2.9921 - val_accuracy: 0.4737
Epoch 2/100
177/177 [==============================] - 0s 539us/step - loss: 0.1135 - accuracy: 0.9605 - val_loss: 3.0083 - val_accuracy: 0.4342
Epoch 3/100
177/177 [==============================] - 0s 547us/step - loss: 0.1480 - accuracy: 0.9605 - val_loss: 2.9106 - val_accuracy: 0.4737
Epoch 4/100
177/177 [==============================] - 0s 558us/step - loss: 0.1608 - accuracy: 0.9548 - val_loss: 2.9860 - val_accuracy: 0.4342
Epoch 5/100
177/177 [==============================] - 0s 535us/step - loss: 0.1937 - accuracy: 0.9661 - val_loss: 3.0510 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 522us/step - loss: 0.1874 - accuracy: 0.9605 - val_loss: 2.9599 - val_accuracy: 0.4474
Epoch 7/100
177/177 [==============================] - 0s 634us/step - loss: 0.1350 -

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.118353). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 1ms/step - loss: 0.4460 - accuracy: 0.9096 - val_loss: 3.6821 - val_accuracy: 0.4737
Epoch 24/100
177/177 [==============================] - 0s 555us/step - loss: 0.2334 - accuracy: 0.9379 - val_loss: 3.4956 - val_accuracy: 0.4474
Epoch 25/100
177/177 [==============================] - 0s 819us/step - loss: 0.1734 - accuracy: 0.9605 - val_loss: 3.1044 - val_accuracy: 0.4868
Epoch 26/100
177/177 [==============================] - 0s 641us/step - loss: 0.1086 - accuracy: 0.9605 - val_loss: 3.0833 - val_accuracy: 0.4342
Epoch 27/100
177/177 [==============================] - 0s 595us/step - loss: 0.1593 - accuracy: 0.9718 - val_loss: 3.0936 - val_accuracy: 0.4605
Epoch 28/100
177/177 [==============================] - 0s 581us/step - loss: 0.1078 - accuracy: 0.9718 - val_loss: 3.1142 - val_accuracy: 0.4868
Epoch 29/100
177/177 [==============================] - 0s 578us/step - loss: 0.1787 - accuracy: 0.9774 - val_loss: 3.2596 - val_accuracy: 

In [114]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 95.86%


In [38]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [39]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [40]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[1.26396850e-01, 8.65422000e-01, 8.18115400e-03],
       [4.10383200e-01, 6.39689600e-04, 5.88977100e-01],
       [2.91246050e-02, 1.85434220e-02, 9.52332000e-01],
       [2.02512030e-01, 7.44713300e-01, 5.27747000e-02],
       [2.76904800e-04, 5.25984650e-02, 9.47124600e-01],
       [1.39941680e-02, 8.16045600e-20, 9.86005800e-01],
       [1.74624660e-02, 7.56512170e-01, 2.26025400e-01],
       [3.03647340e-04, 3.86978000e-02, 9.60998540e-01],
       [6.12981700e-01, 1.89618380e-01, 1.97399960e-01],
       [9.35696200e-01, 7.54482900e-03, 5.67589800e-02],
       [6.10852200e-01, 3.21490470e-01, 6.76573200e-02],
       [9.96480500e-01, 3.93556180e-07, 3.51915560e-03],
       [9.48791800e-01, 3.63506450e-02, 1.48575800e-02],
       [8.99686500e-01, 9.82338560e-02, 2.07962860e-03],
       [6.33348100e-01, 1.14917280e-02, 3.55160150e-01],
       [4.71423100e-01, 4.46810130e-01, 8.17668140e-02],
       [1.48697750e-01, 4.80408220e-01, 3.70894070e-01],
       [2.03739540e-01, 2.02078

In [41]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.6131295952724525

In [42]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.6131295952724525

In [43]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [44]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [45]:
dat4

,id,test
236,SR1129,2
31,CA105,2
155,NRS175,1
92,EUH25,2
122,NRS070,2
...,...,...
235,SR1065,0
24,BCH-SA-09,0
104,Grady1,0
213,NRS254,2


In [46]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [56]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.9811 - accuracy: 0.3503 - val_loss: 1.4700 - val_accuracy: 0.3816
Epoch 2/100
177/177 [==============================] - 0s 501us/step - loss: 1.3690 - accuracy: 0.5085 - val_loss: 1.2163 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 479us/step - loss: 1.0598 - accuracy: 0.5932 - val_loss: 1.0984 - val_accuracy: 0.3158
Epoch 4/100
177/177 [==============================] - 0s 464us/step - loss: 0.9442 - accuracy: 0.6610 - val_loss: 1.2153 - val_accuracy: 0.3158
Epoch 5/100
177/177 [==============================] - 0s 651us/step - loss: 0.8364 - accuracy: 0.6384 - val_loss: 1.2351 - val_accuracy: 0.3158
Epoch 6/100
177/177 [==============================] - 0s 589us/step - loss: 0.7763 - accuracy: 0.6780 - val_loss: 1.3287 - val_accuracy: 0.3158
Epoch 7/100
177/177 [==============================] - 0s 425us/step - loss: 0.7467 - a

Epoch 57/100
177/177 [==============================] - 0s 472us/step - loss: 0.2093 - accuracy: 0.9435 - val_loss: 2.7664 - val_accuracy: 0.4342
Epoch 58/100
177/177 [==============================] - 0s 430us/step - loss: 0.2136 - accuracy: 0.9379 - val_loss: 2.7647 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 382us/step - loss: 0.2024 - accuracy: 0.9435 - val_loss: 2.8130 - val_accuracy: 0.4079
Epoch 60/100
177/177 [==============================] - 0s 495us/step - loss: 0.2052 - accuracy: 0.9492 - val_loss: 2.8254 - val_accuracy: 0.4079
Epoch 61/100
177/177 [==============================] - 0s 374us/step - loss: 0.2031 - accuracy: 0.9435 - val_loss: 2.8566 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 423us/step - loss: 0.1932 - accuracy: 0.9548 - val_loss: 2.9362 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 401us/step - loss: 0.1932 - accuracy: 0.9435 - val_loss: 2.9542 -

In [146]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 154us/step
test accuracy: 46.05%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([2, 2, 0, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2,
       1, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 1, 0,
       1, 2, 0, 1, 1, 2, 2, 2, 0, 2, 0, 1, 0, 2, 1, 0, 0, 2, 0, 0, 2, 2,
       0, 2, 2, 2, 2, 0, 2, 1, 2, 0])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
236,SR1129,2,2
31,CA105,2,2
155,NRS175,1,0
92,EUH25,2,1
122,NRS070,2,2
...,...,...,...
235,SR1065,0,0
24,BCH-SA-09,0,2
104,Grady1,0,1
213,NRS254,2,2


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,0.068795,1.512715e-02,0.916077
1,0.429596,5.621257e-03,0.564782
2,0.997065,2.430247e-03,0.000505
3,0.012187,9.735159e-01,0.014297
4,0.000080,9.668956e-07,0.999919
...,...,...,...
71,0.882353,6.826960e-06,0.117640
72,0.003201,7.025987e-02,0.926539
73,0.143587,7.054675e-01,0.150946
74,0.184970,1.351126e-02,0.801519


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17pST.csv", index = False,
         header=None)

In [150]:
hist4 = model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 433us/step - loss: 0.1443 - accuracy: 0.9548 - val_loss: 2.9793 - val_accuracy: 0.4605
Epoch 2/100
177/177 [==============================] - 0s 450us/step - loss: 0.1452 - accuracy: 0.9661 - val_loss: 3.0322 - val_accuracy: 0.4474
Epoch 3/100
177/177 [==============================] - 0s 406us/step - loss: 0.1292 - accuracy: 0.9605 - val_loss: 3.0450 - val_accuracy: 0.4474
Epoch 4/100
177/177 [==============================] - 0s 406us/step - loss: 0.1574 - accuracy: 0.9605 - val_loss: 3.0648 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 423us/step - loss: 0.1310 - accuracy: 0.9661 - val_loss: 3.0622 - val_accuracy: 0.4474
Epoch 6/100
177/177 [==============================] - 0s 406us/step - loss: 0.1431 - accuracy: 0.9661 - val_loss: 3.0673 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 358us/step - loss: 0.1247 -

Epoch 57/100
177/177 [==============================] - 0s 333us/step - loss: 0.1518 - accuracy: 0.9718 - val_loss: 3.7180 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 365us/step - loss: 0.1418 - accuracy: 0.9831 - val_loss: 3.6369 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 385us/step - loss: 0.1160 - accuracy: 0.9718 - val_loss: 3.6127 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 332us/step - loss: 0.0866 - accuracy: 0.9831 - val_loss: 3.6564 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 347us/step - loss: 0.0974 - accuracy: 0.9774 - val_loss: 3.7425 - val_accuracy: 0.4605
Epoch 62/100
177/177 [==============================] - 0s 351us/step - loss: 0.1017 - accuracy: 0.9774 - val_loss: 3.7597 - val_accuracy: 0.4605
Epoch 63/100
177/177 [==============================] - 0s 352us/step - loss: 0.0746 - accuracy: 0.9718 - val_loss: 3.7988 -

In [151]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 97.49%


In [47]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [48]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [49]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[6.87952700e-02, 1.51271460e-02, 9.16077500e-01],
       [4.29596280e-01, 5.62125700e-03, 5.64782400e-01],
       [9.97065250e-01, 2.43024700e-03, 5.04550500e-04],
       [1.21871010e-02, 9.73515870e-01, 1.42970560e-02],
       [8.00028940e-05, 9.66895600e-07, 9.99919060e-01],
       [1.72127130e-03, 4.25128080e-02, 9.55765900e-01],
       [2.76227180e-02, 6.81554730e-01, 2.90822480e-01],
       [9.99314200e-01, 9.53211600e-06, 6.76300300e-04],
       [9.56757000e-02, 7.27630560e-01, 1.76693810e-01],
       [3.57633980e-05, 9.99866370e-01, 9.79043250e-05],
       [8.71361200e-02, 9.07507800e-01, 5.35616230e-03],
       [1.07976770e-03, 9.98903400e-01, 1.67966700e-05],
       [7.08725440e-03, 9.66506540e-01, 2.64062920e-02],
       [5.15638700e-03, 9.92063600e-01, 2.78006560e-03],
       [8.25230800e-03, 9.91696060e-01, 5.17170050e-05],
       [1.64736140e-01, 7.69136400e-01, 6.61274600e-02],
       [1.19596970e-05, 9.99983800e-01, 4.24389800e-06],
       [8.05527700e-01, 9.77264

In [50]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.5742172289791337

In [51]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.5742172289791337

In [52]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.6078801049039144

In [53]:
np.std(ovos)

0.029610291404617534

In [54]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.6078801049039144

In [55]:
np.std(ovrs)

0.029610291404617534

In [152]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [153]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 47.37%


In [154]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.020804459034607384


In [155]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [156]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 96.21%


In [157]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.017170282


In [76]:
############ Feature selection using lasso ##########

In [56]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [57]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [58]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [59]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [60]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7157
selected features: 511


In [61]:
cols = sel_features.values
cols.reshape((1, -1))

array([[   9,   11,   38,   39,   40,   55,   58,   64,  107,  109,  130,
         147,  149,  182,  187,  222,  225,  227,  235,  267,  270,  273,
         297,  310,  313,  323,  345,  347,  362,  366,  374,  391,  422,
         448,  459,  498,  519,  527,  533,  535,  537,  540,  541,  546,
         551,  590,  601,  626,  632,  637,  639,  690,  742,  745,  749,
         760,  761,  762,  785,  805,  846,  850,  881,  899,  922,  953,
         975,  982,  983,  997, 1031, 1047, 1064, 1066, 1088, 1092, 1109,
        1135, 1162, 1181, 1184, 1193, 1201, 1202, 1208, 1221, 1253, 1255,
        1274, 1278, 1281, 1283, 1287, 1288, 1308, 1321, 1322, 1328, 1400,
        1403, 1414, 1427, 1442, 1461, 1481, 1484, 1493, 1501, 1508, 1518,
        1519, 1547, 1559, 1572, 1598, 1600, 1618, 1619, 1649, 1653, 1654,
        1700, 1734, 1738, 1739, 1746, 1768, 1771, 1801, 1803, 1815, 1830,
        1851, 1859, 1885, 1898, 1902, 1906, 1921, 1946, 1995, 2014, 2026,
        2059, 2088, 2098, 2130, 2131, 

In [62]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA',
       'TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT',
       'TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG',
       'TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA',
       'TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA',
       'TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC',
       'TTTTTGCCGTGGGTAGAAAGTGCTTTTTTATTAATTTTAAAAAAAGCACCAAAAATTTAAATGGAGGTGTCTGAATGTCTGCTATCATTGAAGCTAAAAA',
       'TTTTTCTTTAAATGCTAAAATAATTGATTTCTTTTTATCGTTCGTAAATACGAAATTTTCGTATTCATTACCTAAAAAAATATCATCATATTTAACATTA',
       'TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA',
 

In [63]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [64]:
df_sel['strain'] = X.iloc[:,0]

In [65]:
df_sel

,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT,TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG,TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA,TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA,TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC,TTTTTGCCGTGGGTAGAAAGTGCTTTTTTATTAATTTTAAAAAAAGCACCAAAAATTTAAATGGAGGTGTCTGAATGTCTGCTATCATTGAAGCTAAAAA,TTTTTCTTTAAATGCTAAAATAATTGATTTCTTTTTATCGTTCGTAAATACGAAATTTTCGTATTCATTACCTAAAAAAATATCATCATATTTAACATTA,TTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCGA,TTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAAT,...,group_2498,group_7793,group_4054,group_276,group_178,group_64,ST,CC,pheno,strain
0,1,0,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,5,5,2,107
1,0,1,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,8,8,0,109
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,5,5,2,115
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,5,5,2,120335
4,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,5,5,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,5,5,2,SR4152
249,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,3812,5,1,SR4153
250,0,1,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,5,5,0,SR4155
251,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,5,5,2,SR4156


In [66]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 512) (253,) (253, 513)


In [67]:
df_sel['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [68]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [69]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [70]:
dat5

,strain,test
231,NY360,2
139,NRS113,0
4,120337,2
59,CFBREBSa127,1
96,GA27,2
...,...,...
236,SR1129,2
145,NRS152,2
86,CFBRSa66A,0
28,BCH-SA-13,1


In [71]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [81]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 889us/step - loss: 2.4314 - accuracy: 0.3220 - val_loss: 1.8199 - val_accuracy: 0.3684
Epoch 2/100
177/177 [==============================] - 0s 127us/step - loss: 1.4595 - accuracy: 0.4294 - val_loss: 1.6622 - val_accuracy: 0.2500
Epoch 3/100
177/177 [==============================] - 0s 185us/step - loss: 1.5980 - accuracy: 0.4237 - val_loss: 1.5597 - val_accuracy: 0.2895
Epoch 4/100
177/177 [==============================] - 0s 132us/step - loss: 1.3243 - accuracy: 0.4633 - val_loss: 1.2614 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 143us/step - loss: 1.1129 - accuracy: 0.4746 - val_loss: 1.6519 - val_accuracy: 0.3553
Epoch 6/100
177/177 [==============================] - 0s 123us/step - loss: 1.1636 - accuracy: 0.4802 - val_loss: 1.3694 - val_accuracy: 0.4868
Epoch 7/100
177/177 [==============================] - 0s 131us/step - loss: 0.9847 -

177/177 [==============================] - 0s 89us/step - loss: 0.4980 - accuracy: 0.8249 - val_loss: 1.4310 - val_accuracy: 0.5789
Epoch 58/100
177/177 [==============================] - 0s 82us/step - loss: 0.4880 - accuracy: 0.8362 - val_loss: 1.3191 - val_accuracy: 0.5395
Epoch 59/100
177/177 [==============================] - 0s 126us/step - loss: 0.4524 - accuracy: 0.8531 - val_loss: 1.3353 - val_accuracy: 0.5658
Epoch 60/100
177/177 [==============================] - 0s 104us/step - loss: 0.4630 - accuracy: 0.8418 - val_loss: 1.3438 - val_accuracy: 0.5395
Epoch 61/100
177/177 [==============================] - 0s 87us/step - loss: 0.4575 - accuracy: 0.8588 - val_loss: 1.4033 - val_accuracy: 0.5789
Epoch 62/100
177/177 [==============================] - 0s 73us/step - loss: 0.4468 - accuracy: 0.8588 - val_loss: 1.3434 - val_accuracy: 0.5789
Epoch 63/100
177/177 [==============================] - 0s 74us/step - loss: 0.4818 - accuracy: 0.8305 - val_loss: 1.3538 - val_accuracy: 0.5

In [205]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 77us/step
over-sampling test accuracy: 60.53%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([1, 0, 2, 1, 2, 1, 1, 1, 2, 0, 0, 2, 1, 0, 2, 1, 0, 0, 0, 0, 1, 1,
       2, 2, 0, 2, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 0, 2,
       1, 1, 2, 0, 2, 2, 2, 2, 0, 2, 1, 0, 1, 0, 2, 1, 0, 0, 1, 1, 1, 1,
       0, 2, 0, 0, 0, 2, 2, 0, 1, 2])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
231,NY360,2,1
139,NRS113,0,0
4,120337,2,2
59,CFBREBSa127,1,1
96,GA27,2,2
...,...,...,...
236,SR1129,2,2
145,NRS152,2,2
86,CFBRSa66A,0,0
28,BCH-SA-13,1,1


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,0.319220,0.671314,0.009466
1,0.956011,0.011463,0.032526
2,0.002417,0.007974,0.989609
3,0.048417,0.666007,0.285576
4,0.000127,0.006456,0.993417
...,...,...,...
71,0.083948,0.435341,0.480711
72,0.078950,0.023044,0.898006
73,0.769431,0.165802,0.064767
74,0.168907,0.430691,0.400402


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17pST.csv", index = False,
         header=None)

In [209]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 95us/step - loss: 0.2696 - accuracy: 0.9322 - val_loss: 1.4408 - val_accuracy: 0.5921
Epoch 2/100
177/177 [==============================] - 0s 93us/step - loss: 0.2619 - accuracy: 0.9322 - val_loss: 1.4740 - val_accuracy: 0.5789
Epoch 3/100
177/177 [==============================] - 0s 105us/step - loss: 0.2637 - accuracy: 0.9379 - val_loss: 1.4959 - val_accuracy: 0.5921
Epoch 4/100
177/177 [==============================] - 0s 93us/step - loss: 0.2576 - accuracy: 0.9322 - val_loss: 1.4944 - val_accuracy: 0.5789
Epoch 5/100
177/177 [==============================] - 0s 100us/step - loss: 0.2590 - accuracy: 0.9209 - val_loss: 1.4910 - val_accuracy: 0.5921
Epoch 6/100
177/177 [==============================] - 0s 119us/step - loss: 0.2512 - accuracy: 0.9379 - val_loss: 1.4999 - val_accuracy: 0.5789
Epoch 7/100
177/177 [==============================] - 0s 98us/step - loss: 0.2542 - acc

177/177 [==============================] - 0s 91us/step - loss: 0.1370 - accuracy: 0.9718 - val_loss: 1.6571 - val_accuracy: 0.6053
Epoch 58/100
177/177 [==============================] - 0s 125us/step - loss: 0.1334 - accuracy: 0.9718 - val_loss: 1.6543 - val_accuracy: 0.6184
Epoch 59/100
177/177 [==============================] - 0s 80us/step - loss: 0.1360 - accuracy: 0.9718 - val_loss: 1.6695 - val_accuracy: 0.6184
Epoch 60/100
177/177 [==============================] - 0s 89us/step - loss: 0.1234 - accuracy: 0.9774 - val_loss: 1.6799 - val_accuracy: 0.6184
Epoch 61/100
177/177 [==============================] - 0s 86us/step - loss: 0.1245 - accuracy: 0.9774 - val_loss: 1.6399 - val_accuracy: 0.6053
Epoch 62/100
177/177 [==============================] - 0s 105us/step - loss: 0.1168 - accuracy: 0.9774 - val_loss: 1.6676 - val_accuracy: 0.6053
Epoch 63/100
177/177 [==============================] - 0s 108us/step - loss: 0.1184 - accuracy: 0.9718 - val_loss: 1.6857 - val_accuracy: 0.

In [210]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 95.59%


In [72]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [73]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [74]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[3.19220040e-01, 6.71313640e-01, 9.46632100e-03],
       [9.56010760e-01, 1.14627555e-02, 3.25264900e-02],
       [2.41723680e-03, 7.97393450e-03, 9.89608800e-01],
       [4.84172200e-02, 6.66006860e-01, 2.85575960e-01],
       [1.27318750e-04, 6.45613340e-03, 9.93416550e-01],
       [2.18570630e-01, 6.87941300e-01, 9.34880700e-02],
       [1.52052500e-02, 8.52544900e-01, 1.32249860e-01],
       [2.41774190e-01, 7.52811400e-01, 5.41445240e-03],
       [3.78184320e-01, 4.66579080e-07, 6.21815200e-01],
       [6.69107740e-01, 3.13014150e-01, 1.78781170e-02],
       [5.38035600e-01, 4.14930940e-01, 4.70335400e-02],
       [2.69195300e-04, 3.92985400e-07, 9.99730400e-01],
       [3.65392630e-01, 5.73541100e-01, 6.10662070e-02],
       [8.24086070e-01, 5.34322820e-02, 1.22481660e-01],
       [1.50150130e-03, 3.01263350e-02, 9.68372170e-01],
       [4.99642450e-02, 8.85122360e-01, 6.49134600e-02],
       [3.89094200e-01, 3.38006440e-01, 2.72899420e-01],
       [9.93347800e-01, 2.88456

In [75]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.7247446116493735

In [76]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.7247446116493735

In [77]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [78]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [79]:
dat6

,strain,test
44,CFBREBSa104,1
170,NRS199,2
197,NRS233,1
238,SR1746,0
172,NRS202,2
...,...,...
92,EUH25,2
217,NRS260,0
75,CFBRSa25,0
156,NRS177,1


In [80]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [81]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 717us/step - loss: 6.7044 - accuracy: 0.3333 - val_loss: 3.2322 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 188us/step - loss: 3.1536 - accuracy: 0.4237 - val_loss: 2.0982 - val_accuracy: 0.4605
Epoch 3/100
177/177 [==============================] - 0s 279us/step - loss: 1.9138 - accuracy: 0.5028 - val_loss: 1.4340 - val_accuracy: 0.3421
Epoch 4/100
177/177 [==============================] - 0s 324us/step - loss: 1.4327 - accuracy: 0.5311 - val_loss: 1.3462 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 242us/step - loss: 1.0663 - accuracy: 0.5593 - val_loss: 1.2081 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 280us/step - loss: 0.9725 - accuracy: 0.5989 - val_loss: 1.1119 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 222us/step - loss: 0.9453 -

Epoch 57/100
177/177 [==============================] - 0s 318us/step - loss: 0.5434 - accuracy: 0.8475 - val_loss: 1.0852 - val_accuracy: 0.5658
Epoch 58/100
177/177 [==============================] - 0s 255us/step - loss: 0.4616 - accuracy: 0.8644 - val_loss: 1.0744 - val_accuracy: 0.5921
Epoch 59/100
177/177 [==============================] - 0s 302us/step - loss: 0.4416 - accuracy: 0.8927 - val_loss: 1.0354 - val_accuracy: 0.5789
Epoch 60/100
177/177 [==============================] - 0s 210us/step - loss: 0.4085 - accuracy: 0.8757 - val_loss: 1.0776 - val_accuracy: 0.6053
Epoch 61/100
177/177 [==============================] - 0s 185us/step - loss: 0.4759 - accuracy: 0.8757 - val_loss: 1.0365 - val_accuracy: 0.6053
Epoch 62/100
177/177 [==============================] - 0s 212us/step - loss: 0.3983 - accuracy: 0.8983 - val_loss: 1.0144 - val_accuracy: 0.6316
Epoch 63/100
177/177 [==============================] - 0s 313us/step - loss: 0.4215 - accuracy: 0.8927 - val_loss: 1.0193 -

In [242]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 131us/step
test accuracy: 59.21%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([2, 2, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 0, 2, 2, 2, 0, 1, 2, 0, 1, 2, 1, 2,
       1, 2, 2, 1, 0, 1, 2, 0, 0, 1, 2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 0, 1,
       0, 0, 0, 0, 2, 1, 0, 0, 1, 2])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
44,CFBREBSa104,1,2
170,NRS199,2,2
197,NRS233,1,0
238,SR1746,0,1
172,NRS202,2,2
...,...,...,...
92,EUH25,2,1
217,NRS260,0,0
75,CFBRSa25,0,0
156,NRS177,1,1


In [99]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [100]:
dat_proba6

,0,1,2
0,0.187464,0.029691,0.782845
1,0.231965,0.376493,0.391541
2,0.663481,0.095792,0.240727
3,0.171868,0.519125,0.309007
4,0.034450,0.119096,0.846455
...,...,...,...
71,0.373144,0.625452,0.001403
72,0.976010,0.013766,0.010224
73,0.672251,0.247505,0.080244
74,0.016700,0.972865,0.010435


In [101]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [102]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17pST.csv", index = False,
         header=None)

In [246]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 196us/step - loss: 0.4251 - accuracy: 0.8983 - val_loss: 0.9886 - val_accuracy: 0.5921
Epoch 2/100
177/177 [==============================] - 0s 236us/step - loss: 0.3680 - accuracy: 0.9040 - val_loss: 1.0817 - val_accuracy: 0.6316
Epoch 3/100
177/177 [==============================] - 0s 230us/step - loss: 0.3978 - accuracy: 0.8870 - val_loss: 0.9710 - val_accuracy: 0.5921
Epoch 4/100
177/177 [==============================] - 0s 186us/step - loss: 0.3575 - accuracy: 0.9153 - val_loss: 1.0696 - val_accuracy: 0.6053
Epoch 5/100
177/177 [==============================] - 0s 192us/step - loss: 0.3608 - accuracy: 0.9096 - val_loss: 1.0122 - val_accuracy: 0.6184
Epoch 6/100
177/177 [==============================] - 0s 180us/step - loss: 0.3479 - accuracy: 0.9153 - val_loss: 1.0307 - val_accuracy: 0.5921
Epoch 7/100
177/177 [==============================] - 0s 175us/step - loss: 0.4668 -

Epoch 57/100
177/177 [==============================] - 0s 1ms/step - loss: 0.2477 - accuracy: 0.9322 - val_loss: 1.0951 - val_accuracy: 0.6447
Epoch 58/100
177/177 [==============================] - 0s 395us/step - loss: 0.2436 - accuracy: 0.9379 - val_loss: 1.0392 - val_accuracy: 0.6316
Epoch 59/100
177/177 [==============================] - 0s 767us/step - loss: 0.2809 - accuracy: 0.9209 - val_loss: 1.0675 - val_accuracy: 0.6053
Epoch 60/100
177/177 [==============================] - 0s 303us/step - loss: 0.2004 - accuracy: 0.9435 - val_loss: 1.0798 - val_accuracy: 0.5921
Epoch 61/100
177/177 [==============================] - 0s 389us/step - loss: 0.2279 - accuracy: 0.9548 - val_loss: 1.0785 - val_accuracy: 0.6184
Epoch 62/100
177/177 [==============================] - 0s 232us/step - loss: 0.2649 - accuracy: 0.9548 - val_loss: 1.0902 - val_accuracy: 0.6053
Epoch 63/100
177/177 [==============================] - 0s 373us/step - loss: 0.2645 - accuracy: 0.9492 - val_loss: 1.1691 - v

In [247]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 92.94%


In [82]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [83]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [84]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.87463980e-01, 2.96913940e-02, 7.82844660e-01],
       [2.31965440e-01, 3.76493360e-01, 3.91541180e-01],
       [6.63481240e-01, 9.57917350e-02, 2.40727070e-01],
       [1.71868160e-01, 5.19125100e-01, 3.09006780e-01],
       [3.44496400e-02, 1.19095650e-01, 8.46454700e-01],
       [5.44801300e-03, 2.19516890e-01, 7.75035100e-01],
       [7.66627200e-02, 1.34719670e-01, 7.88617700e-01],
       [7.56010800e-01, 1.85391750e-01, 5.85975200e-02],
       [4.00522900e-02, 9.21841440e-01, 3.81062900e-02],
       [5.85080860e-01, 3.81434230e-01, 3.34849430e-02],
       [1.83461640e-01, 6.70618800e-01, 1.45919640e-01],
       [2.89404860e-02, 2.51123520e-01, 7.19936000e-01],
       [3.13181760e-01, 5.69331470e-01, 1.17486745e-01],
       [5.34935160e-03, 8.13380900e-03, 9.86516830e-01],
       [8.00847500e-01, 1.30704050e-01, 6.84485200e-02],
       [7.59959440e-02, 5.69333140e-02, 8.67070800e-01],
       [3.00342100e-01, 5.56674700e-01, 1.42983240e-01],
       [7.30021400e-01, 1.14089

In [85]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.7380608809180238

In [86]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.7380608809180238

In [87]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [88]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [89]:
dat7

,strain,test
222,NRS271,1
242,SR2852,2
34,CA39,2
221,NRS266,1
230,NY356,2
...,...,...
105,MN055,2
67,CFBRSa04,0
235,SR1065,0
190,NRS224,1


In [90]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [107]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [108]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 822us/step - loss: 8.4130 - accuracy: 0.3503 - val_loss: 7.1595 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 94us/step - loss: 7.0588 - accuracy: 0.4124 - val_loss: 6.2712 - val_accuracy: 0.3947
Epoch 3/100
177/177 [==============================] - 0s 118us/step - loss: 5.6857 - accuracy: 0.4520 - val_loss: 5.4865 - val_accuracy: 0.3684
Epoch 4/100
177/177 [==============================] - 0s 185us/step - loss: 4.6552 - accuracy: 0.4181 - val_loss: 4.7701 - val_accuracy: 0.4079
Epoch 5/100
177/177 [==============================] - 0s 145us/step - loss: 3.6774 - accuracy: 0.4463 - val_loss: 4.1792 - val_accuracy: 0.3816
Epoch 6/100
177/177 [==============================] - 0s 244us/step - loss: 2.7574 - accuracy: 0.4576 - val_loss: 3.7470 - val_accuracy: 0.3816
Epoch 7/100
177/177 [==============================] - 0s 139us/step - loss: 2.2263 - 

Epoch 57/100
177/177 [==============================] - 0s 88us/step - loss: 0.4702 - accuracy: 0.8475 - val_loss: 1.2733 - val_accuracy: 0.4737
Epoch 58/100
177/177 [==============================] - 0s 83us/step - loss: 0.4638 - accuracy: 0.8475 - val_loss: 1.4268 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 96us/step - loss: 0.4563 - accuracy: 0.8475 - val_loss: 1.3689 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 74us/step - loss: 0.4270 - accuracy: 0.8531 - val_loss: 1.2992 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 83us/step - loss: 0.4189 - accuracy: 0.8701 - val_loss: 1.2954 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 80us/step - loss: 0.4247 - accuracy: 0.8814 - val_loss: 1.3272 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 86us/step - loss: 0.4092 - accuracy: 0.8701 - val_loss: 1.4068 - val_ac

In [299]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 73us/step
test accuracy: 46.05%


In [110]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 1, 2, 0, 0, 1, 2, 2, 1, 2, 1, 2, 0, 1, 1, 2, 1, 1, 2, 2, 1,
       0, 0, 0, 2, 0, 0, 1, 2, 1, 0, 0, 2, 1, 2, 0, 1, 0, 2, 2, 2, 1, 0,
       2, 0, 1, 0, 0, 2, 1, 2, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 0, 1, 0, 0, 2, 0, 0, 2, 1])

In [111]:
dat7['pred'] = pred7
dat7

,strain,test,pred
222,NRS271,1,1
242,SR2852,2,2
34,CA39,2,1
221,NRS266,1,2
230,NY356,2,0
...,...,...,...
105,MN055,2,2
67,CFBRSa04,0,0
235,SR1065,0,0
190,NRS224,1,2


In [112]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [113]:
dat_proba7

,0,1,2
0,0.120862,0.863844,0.015294
1,0.014857,0.402346,0.582797
2,0.023940,0.611152,0.364908
3,0.156154,0.340488,0.503358
4,0.592273,0.380877,0.026850
...,...,...,...
71,0.041429,0.026486,0.932085
72,0.940746,0.055868,0.003387
73,0.869274,0.036650,0.094076
74,0.289750,0.000010,0.710240


In [114]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [115]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17p.csv", index = False,
         header=None)

In [303]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 98us/step - loss: 0.2622 - accuracy: 0.9322 - val_loss: 1.4312 - val_accuracy: 0.5000
Epoch 2/100
177/177 [==============================] - 0s 90us/step - loss: 0.2570 - accuracy: 0.9435 - val_loss: 1.4700 - val_accuracy: 0.4868
Epoch 3/100
177/177 [==============================] - 0s 109us/step - loss: 0.2531 - accuracy: 0.9379 - val_loss: 1.5152 - val_accuracy: 0.5000
Epoch 4/100
177/177 [==============================] - 0s 119us/step - loss: 0.2454 - accuracy: 0.9435 - val_loss: 1.4860 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 128us/step - loss: 0.2447 - accuracy: 0.9548 - val_loss: 1.4758 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 126us/step - loss: 0.2385 - accuracy: 0.9435 - val_loss: 1.5136 - val_accuracy: 0.5132
Epoch 7/100
177/177 [==============================] - 0s 109us/step - loss: 0.2363 - a

In [304]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 97.02%


In [91]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [92]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [93]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[1.2086153e-01, 8.6384410e-01, 1.5294372e-02],
       [1.4856748e-02, 4.0234596e-01, 5.8279730e-01],
       [2.3940066e-02, 6.1115150e-01, 3.6490850e-01],
       [1.5615402e-01, 3.4048802e-01, 5.0335795e-01],
       [5.9227330e-01, 3.8087663e-01, 2.6850142e-02],
       [9.7391200e-01, 5.5985650e-03, 2.0489430e-02],
       [1.0621916e-02, 8.7807080e-01, 1.1130733e-01],
       [7.9336450e-02, 2.4120276e-01, 6.7946076e-01],
       [9.8634504e-02, 4.0473092e-01, 4.9663457e-01],
       [6.6578540e-02, 9.3153780e-01, 1.8835821e-03],
       [2.5920755e-01, 4.2917213e-04, 7.4036330e-01],
       [2.4300893e-01, 4.9691424e-01, 2.6007676e-01],
       [4.3629950e-04, 4.4810330e-02, 9.5475334e-01],
       [6.8063915e-01, 3.2407406e-03, 3.1612010e-01],
       [1.1110698e-01, 8.7137514e-01, 1.7517816e-02],
       [2.5254510e-01, 4.3542585e-01, 3.1202910e-01],
       [2.4866147e-02, 2.3286150e-02, 9.5184770e-01],
       [4.2241812e-01, 4.3617767e-01, 1.4140424e-01],
       [2.1103641e-02, 7.795

In [94]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.6571210975972881

In [95]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.6571210975972881

In [96]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [97]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [98]:
dat8

,strain,test
131,NRS104,0
123,NRS071,0
124,NRS072,1
27,BCH-SA-12,0
57,CFBREBSa125,2
...,...,...
170,NRS199,2
32,CA11,2
212,NRS253,1
141,NRS119,2


In [99]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [100]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [121]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 754us/step - loss: 2.4099 - accuracy: 0.3785 - val_loss: 1.2550 - val_accuracy: 0.4211
Epoch 2/100
177/177 [==============================] - 0s 154us/step - loss: 1.4272 - accuracy: 0.3898 - val_loss: 1.3649 - val_accuracy: 0.3684
Epoch 3/100
177/177 [==============================] - 0s 150us/step - loss: 1.1568 - accuracy: 0.4915 - val_loss: 1.1453 - val_accuracy: 0.4605
Epoch 4/100
177/177 [==============================] - 0s 180us/step - loss: 1.0748 - accuracy: 0.4915 - val_loss: 1.0719 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 136us/step - loss: 1.0663 - accuracy: 0.4746 - val_loss: 1.0592 - val_accuracy: 0.4342
Epoch 6/100
177/177 [==============================] - 0s 165us/step - loss: 1.0389 - accuracy: 0.5311 - val_loss: 1.0515 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 137us/step - loss: 1.0041 -

Epoch 57/100
177/177 [==============================] - 0s 187us/step - loss: 0.4423 - accuracy: 0.8644 - val_loss: 1.1404 - val_accuracy: 0.5526
Epoch 58/100
177/177 [==============================] - 0s 181us/step - loss: 0.4103 - accuracy: 0.8644 - val_loss: 1.1842 - val_accuracy: 0.5395
Epoch 59/100
177/177 [==============================] - 0s 189us/step - loss: 0.3671 - accuracy: 0.8927 - val_loss: 1.1730 - val_accuracy: 0.5526
Epoch 60/100
177/177 [==============================] - 0s 286us/step - loss: 0.4577 - accuracy: 0.8757 - val_loss: 1.1938 - val_accuracy: 0.5658
Epoch 61/100
177/177 [==============================] - 0s 166us/step - loss: 0.3544 - accuracy: 0.8870 - val_loss: 1.3491 - val_accuracy: 0.5132
Epoch 62/100
177/177 [==============================] - 0s 240us/step - loss: 0.4367 - accuracy: 0.8531 - val_loss: 1.2434 - val_accuracy: 0.5526
Epoch 63/100
177/177 [==============================] - 0s 160us/step - loss: 0.3645 - accuracy: 0.8870 - val_loss: 1.2038 -

In [341]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 175us/step
over-sampling test accuracy: 57.89%


In [101]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [102]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [103]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017presabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[9.01472100e-03, 9.50091660e-01, 4.08937200e-02],
       [2.00832220e-03, 9.76960660e-01, 2.10310780e-02],
       [4.08984800e-01, 5.22061060e-02, 5.38809060e-01],
       [3.60743670e-01, 5.98007700e-01, 4.12486500e-02],
       [3.14415150e-03, 5.35487000e-02, 9.43307160e-01],
       [9.23743700e-02, 6.57633100e-01, 2.49992500e-01],
       [8.99010200e-01, 9.99789400e-02, 1.01097940e-03],
       [4.03003100e-02, 5.07762300e-01, 4.51937350e-01],
       [1.20078065e-01, 7.91521300e-01, 8.84006400e-02],
       [8.97442700e-01, 7.96436100e-02, 2.29135920e-02],
       [2.25779010e-01, 6.76036360e-01, 9.81846450e-02],
       [2.24848740e-02, 4.25320660e-01, 5.52194500e-01],
       [2.53055970e-02, 1.81754810e-01, 7.92939600e-01],
       [7.31713530e-01, 8.01618500e-02, 1.88124630e-01],
       [9.06770770e-01, 7.96590150e-02, 1.35702130e-02],
       [1.36409210e-03, 1.84255170e-02, 9.80210360e-01],
       [4.76679360e-04, 9.61809700e-01, 3.77136200e-02],
       [2.94208400e-02, 1.91278

In [104]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.7055818960580865

In [105]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.7055818960580865

In [106]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.706377121555693

In [107]:
np.std(ovos2)

0.03069207381171293

In [108]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.706377121555693

In [109]:
np.std(ovrs2)

0.03069207381171293

In [123]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([1, 1, 2, 1, 2, 1, 0, 1, 1, 0, 1, 2, 2, 0, 0, 2, 1, 2, 0, 1, 2, 2,
       0, 1, 0, 2, 2, 1, 2, 2, 2, 0, 0, 1, 1, 1, 1, 2, 0, 2, 0, 2, 0, 2,
       0, 2, 1, 0, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 1, 0, 1, 0, 2, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0])

In [124]:
dat8['pred'] = pred8
dat8

,strain,test,pred
131,NRS104,0,1
123,NRS071,0,1
124,NRS072,1,2
27,BCH-SA-12,0,1
57,CFBREBSa125,2,2
...,...,...,...
170,NRS199,2,1
32,CA11,2,0
212,NRS253,1,1
141,NRS119,2,1


In [125]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [126]:
dat_proba8

,0,1,2
0,0.009015,0.950092,0.040894
1,0.002008,0.976961,0.021031
2,0.408985,0.052206,0.538809
3,0.360744,0.598008,0.041249
4,0.003144,0.053549,0.943307
...,...,...,...
71,0.368220,0.481331,0.150449
72,0.766999,0.225874,0.007127
73,0.069543,0.731683,0.198774
74,0.007361,0.661300,0.331340


In [127]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [128]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17pST.csv", index = False,
         header=None)

In [345]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 129us/step - loss: 0.2303 - accuracy: 0.9379 - val_loss: 1.5480 - val_accuracy: 0.5921
Epoch 2/100
177/177 [==============================] - 0s 104us/step - loss: 0.3055 - accuracy: 0.9153 - val_loss: 1.6071 - val_accuracy: 0.5658
Epoch 3/100
177/177 [==============================] - 0s 147us/step - loss: 0.3317 - accuracy: 0.9322 - val_loss: 1.7777 - val_accuracy: 0.5658
Epoch 4/100
177/177 [==============================] - 0s 153us/step - loss: 0.3369 - accuracy: 0.9096 - val_loss: 1.6167 - val_accuracy: 0.5789
Epoch 5/100
177/177 [==============================] - 0s 145us/step - loss: 0.2309 - accuracy: 0.9266 - val_loss: 1.8766 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 128us/step - loss: 0.3105 - accuracy: 0.9209 - val_loss: 1.7281 - val_accuracy: 0.5526
Epoch 7/100
177/177 [==============================] - 0s 138us/step - loss: 0.1994 -

Epoch 57/100
177/177 [==============================] - 0s 141us/step - loss: 0.1321 - accuracy: 0.9774 - val_loss: 1.9415 - val_accuracy: 0.5395
Epoch 58/100
177/177 [==============================] - 0s 149us/step - loss: 0.1315 - accuracy: 0.9774 - val_loss: 1.9651 - val_accuracy: 0.5395
Epoch 59/100
177/177 [==============================] - 0s 140us/step - loss: 0.1058 - accuracy: 0.9718 - val_loss: 2.0262 - val_accuracy: 0.5395
Epoch 60/100
177/177 [==============================] - 0s 128us/step - loss: 0.1105 - accuracy: 0.9831 - val_loss: 1.9486 - val_accuracy: 0.5526
Epoch 61/100
177/177 [==============================] - 0s 132us/step - loss: 0.1557 - accuracy: 0.9605 - val_loss: 2.0070 - val_accuracy: 0.5526
Epoch 62/100
177/177 [==============================] - 0s 133us/step - loss: 0.1249 - accuracy: 0.9661 - val_loss: 1.9836 - val_accuracy: 0.5395
Epoch 63/100
177/177 [==============================] - 0s 122us/step - loss: 0.1245 - accuracy: 0.9718 - val_loss: 2.0204 -

In [346]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 95.89%


In [347]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [348]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 55.92%


In [349]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.05773003194789586


In [350]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [351]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 95.36%


In [352]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.014935704
